In [2]:
# Installing Necessary Libraries
pip install openai-whisper PyQt5 sounddevice numpy torch


SyntaxError: invalid syntax (1026907268.py, line 2)

In [ ]:
pip install ffmpeg-python


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Importing necessary Libraries  and PYQT for making Interface
import sys
import whisper
import sounddevice as sd
import numpy as np
import soundfile as sf
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QLabel, QVBoxLayout

In [ ]:
# Load the Whisper model
model = whisper.load_model("base")  # Choose from 'tiny', 'base', 'small', 'medium', 'large'


In [ ]:
class SpeechToTextApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.label = QLabel('Click "Start" to begin recording your voice.', self)
        self.result_label = QLabel('', self)
        self.start_button = QPushButton('Start', self)
        self.stop_button = QPushButton('Stop', self)

        self.start_button.clicked.connect(self.start_recording)
        self.stop_button.clicked.connect(self.stop_recording)
        self.stop_button.setEnabled(False)

        layout = QVBoxLayout()
        layout.addWidget(self.label)
        layout.addWidget(self.start_button)
        layout.addWidget(self.stop_button)
        layout.addWidget(self.result_label)
        self.setLayout(layout)

        self.setWindowTitle('Real-Time Speech to Text using Whisper')
        self.setGeometry(100, 100, 500, 250)

        self.audio_data = []
        self.samplerate = 16000

    def callback(self, indata, frames, time, status):
        if status:
            print(status)
        self.audio_data.append(indata.copy())

    def start_recording(self):
        self.label.setText("Recording... Speak Now.")
        self.audio_data = []
        self.start_button.setEnabled(False)
        self.stop_button.setEnabled(True)
        self.stream = sd.InputStream(callback=self.callback, channels=1, samplerate=self.samplerate)
        self.stream.start()

    def stop_recording(self):
        self.stream.stop()
        self.start_button.setEnabled(True)
        self.stop_button.setEnabled(False)
        self.label.setText("Processing audio...")
        self.process_audio()

    def process_audio(self):
        audio_data = np.concatenate(self.audio_data, axis=0).flatten()
        audio_data = (audio_data * 32767).astype(np.int16)  # Convert to int16 format

        # Save as temporary WAV for Whisper input
        sf.write("temp_audio.mp3", audio_data, self.samplerate)

        # Transcribe using Whisper
        result = model.transcribe("temp_audio.mp3", language="hi")  # Detect Hindi or any other language
        transcription = result['text']

        self.result_label.setText(f"Transcription: {transcription}")
        self.label.setText("Recording completed. Press Start for another.")




In [ ]:
# Run the application
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = SpeechToTextApp()
    ex.show()
    sys.exit(app.exec_())

NameError: name 'QApplication' is not defined